# Summary graph for learning curves 

In [30]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go

In [31]:
# Loading datasets
future_cost_data = pd.read_csv('data/Cost_data_summary.csv', names = ['tech', 'year', 'cost_ratio', 'c_inv','installed_cap','annual_prod'], header=0)
hist_cost_data = pd.read_csv('data/Historical_data_summary.csv', names = ['tech', 'year', 'c_inv','installed_cap'], header=0)
hist_cost_electrolysis = pd.read_csv('data/Historical_electrolysis.csv', names = ['tech', 'year', 'c_inv','installed_cap','annual_prod'], header=0)
hist_cost_carbon_capture = pd.read_csv('data/Historical_carbon_capture.csv', names = ['tech', 'year', 'c_inv','installed_cap'], header=0)
hist_cost_gshp = pd.read_csv('data/Historical_gshp.csv', names = ['tech', 'year', 'c_inv','installed_cap'], header=0)

In [32]:
future_cost_data

,tech,year,cost_ratio,c_inv,installed_cap,annual_prod
0,PV,2018,1.000,2704.85,58.00,NaN
1,PV,2019,0.927,2506.80,74.00,NaN
2,PV,2020,0.903,2442.72,93.00,NaN
3,PV,2021,0.692,1872.68,150.67,NaN
4,PV,2022,0.606,1639.53,208.35,NaN
...,...,...,...,...,...,...
448,WIND_OFFSHORE,2046,0.270,1213.00,1164.15,NaN
449,WIND_OFFSHORE,2047,0.266,1198.00,1214.98,NaN
450,WIND_OFFSHORE,2048,0.263,1183.00,1265.81,NaN
451,WIND_OFFSHORE,2049,0.260,1170.00,1316.64,NaN


In [33]:
hist_cost_data

,tech,year,c_inv,installed_cap
0,ONSHORE_WIND,2010,1914.0,178.0
1,ONSHORE_WIND,2011,1883.0,216.0
2,ONSHORE_WIND,2012,1937.0,261.0
3,ONSHORE_WIND,2013,1797.0,292.0
4,ONSHORE_WIND,2014,1745.0,340.0
...,...,...,...,...
122,HYDRO_TOTAL,2016,1753.0,1245.0
123,HYDRO_TOTAL,2017,1799.0,1269.0
124,HYDRO_TOTAL,2018,1410.0,1292.0
125,HYDRO_TOTAL,2019,1674.0,1305.0


In [34]:
future_cost_data.tech.unique()

array(['PV', 'WIND', 'WOOD_GASIFICATION', 'SOEC_ELECTROLYSIS',
       'PEM_ELECTROLYSIS', 'ALKALINE_ELECTROLYSIS', 'DEC_HP_ELEC',
       'CARBON_CAPTURE', 'CARBON_SEQUSTRATION', 'DEC_SOLAR', 'HYDRO_DAM',
       'HYDRO_RIVER', 'DHN_HP_ELEC', 'WIND_OFFSHORE'], dtype=object)

In [35]:
hist_cost_data.tech.unique()

array(['ONSHORE_WIND', 'RES_PV', 'DEC_HP_ELEC', 'DEC_SOLAR',
       'DHN_HP_ELEC', 'OFFSHORE_WIND', 'UTILITY_PV', 'COMMERCIAL_PV',
       'GEOTHERMAL', 'HYDRO_TOTAL'], dtype=object)

# Parameters for the graph

In [36]:
# graph
marker_size = 6
line_width = 1.5
years_font_size = 10

# colours 
colour_main_axis = "rgb(10,10,10)"
colour_second_axis = "rgb(10,10,10)"
colour_carbon_capture = "rgb(105,105,105)"
colour_onshore_wind = 'rgb(0,0,205)'
colour_offshore_wind = 'rgb(30,144,255)'
colour_res_pv = 'rgb(255,215,0)'
colour_csp_solar = 'rgb(255,140,0)'
colour_gshp = 'rgb(50,205,50)'
colour_ashp = 'rgb(0,128,128)'
colour_soec = 'rgb(139,0,139)'
colour_alk = 'rgb(210,105,30)'
colour_pem = 'rgb(178,34,34)'

## Onshore Wind

In [37]:
# Historical data
hist_cost = hist_cost_data[hist_cost_data.tech == 'ONSHORE_WIND'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'WIND'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a+b*log(Q)
a = 8.796934102 
b = -0.23372041642

In [38]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

fig = go.Figure()

fig.update_layout(
    title=dict(text="",y=1),
    xaxis1=dict(title="Cumulative Capacity [GW]"),
    xaxis2=dict(title="CO<sub>2</sub> capture capacity [Mtpa]",
                title_font_color = colour_second_axis,
                anchor="y",
                overlaying="x",
                side="top",
                #matches='x',
                color=colour_carbon_capture,automargin=True),
    yaxis1=dict(title="Investment Cost [USD<sub>2018</sub>/kW]"),
    yaxis2=dict(title="Cost of CO<sub>2</sub> capture [USD<sub>2018</sub>/tCO<sub>2</sub>]",
                title_font_color = colour_second_axis,
                anchor="x",
                overlaying="y",
                side="right",
                #matches='y',
                color=colour_carbon_capture,automargin=True),
    font=dict(
        family="Times New Roman",
        size=16,
        color=colour_main_axis),
    legend=dict(x=1.20)
)

fig.update_xaxes(type="log",title_font_family="Times New Roman",showgrid=True)
fig.update_yaxes(type="log",title_font_family="Times New Roman",showgrid=True)

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         legendgroup="group",
                         legendgrouptitle_text="Power capacity",
                         legendgrouptitle_font_color=colour_main_axis,
                         name='Onshore Wind - Prospective',
                         marker = dict(color=colour_onshore_wind, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         legendgroup="group",
                         name='Onshore Wind - Historical',
                         marker = dict(color=colour_onshore_wind, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         legendgroup="group",
                         name='Onshore Wind - Learning Curve',
                         line=dict(color=colour_onshore_wind, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

# Offshore Wind

In [39]:
# Historical data
hist_cost = hist_cost_data[hist_cost_data.tech == 'OFFSHORE_WIND'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'WIND_OFFSHORE'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a+b*log(Q)
a = 9.176268277
b = -0.29401459277

In [40]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='Offshore Wind - Prospective',
                         marker = dict(color=colour_offshore_wind, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='Offshore Wind - Historical',
                         marker = dict(color=colour_offshore_wind, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='Offshore Wind - Learning Curve',
                         line=dict(color=colour_offshore_wind, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

## Residential PV

In [41]:
# Historical data
hist_cost = hist_cost_data[hist_cost_data.tech == 'RES_PV'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'PV'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a+b*log(Q)
a = 9.5926123108  
b = -0.41025634487

In [42]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='Residential PV - Prospective',
                         marker = dict(color=colour_res_pv, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         textfont_size=years_font_size,
                         mode='markers+text',
                         marker_symbol='circle',
                         name='Residential PV - Historical',
                         marker = dict(color=colour_res_pv, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='Residential PV - Learning Curve',
                         line=dict(color=colour_res_pv, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

## CSP Solar (i.e. decentralised solar)

In [43]:
# Historical data
hist_cost = hist_cost_data[hist_cost_data.tech == 'DEC_SOLAR'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'DEC_SOLAR'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a*log(Q) + b*log(Q)^2 + c
a = -0.054974571233  
b = -0.065855458185
c = 9.1025403877

In [44]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a*np.log(x_reg)+b*(np.log(x_reg))**2+c)

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='CSP - Prospective',
                         marker = dict(color=colour_csp_solar, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='CSP - Historical',
                         marker = dict(color=colour_csp_solar, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='CSP - Learning Curve',
                         line=dict(color=colour_csp_solar, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

## DHN heat pumps

In [45]:
# Historical data
hist_cost = hist_cost_gshp[hist_cost_gshp.tech == 'DHN_HP_ELEC'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'DHN_HP_ELEC'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a+b*log(Q)
a = 8.3955561938  
b = -0.59713477411

In [46]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

first_prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2018)].installed_cap)
first_prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2018)].c_inv)
first_prosp_year = 2018

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='Ground Source Heat Pumps - Prospective',
                         marker = dict(color=colour_gshp, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='Ground Source Heat Pumps - Historical',
                         marker = dict(color=colour_gshp, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='Ground Source Heat Pumps - Learning Curve',
                         line=dict(color=colour_gshp, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(first_prosp_x), y=np.log10(first_prosp_y),
            text=str(first_prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

## Decentralised Heat Pumps

In [47]:
# Historical data
hist_cost = hist_cost_data[hist_cost_data.tech == 'DEC_HP_ELEC'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'DEC_HP_ELEC'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex)==a+b*log(Q)
a = 7.89738474154832
b = -0.59713477411

In [48]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='Decentralized Heat Pumps - Prospective',
                         marker = dict(color=colour_ashp, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='Decentralized Heat Pumps - Historical',
                         marker = dict(color=colour_ashp, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='Decentralized Heat Pumps - Learning Curve',
                         line=dict(color=colour_ashp, width=line_width)))

fig.add_annotation(x=np.log10(first_x), y=np.log10(first_y),
            text=str(first_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(last_x), y=np.log10(last_y),
            text=str(last_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
            text=str(prosp_year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

## SOEC Electrolysis

In [49]:
# Historical data
hist_cost = hist_cost_electrolysis[hist_cost_electrolysis.tech == 'SOEC_ELECTROLYSIS'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'SOEC_ELECTROLYSIS'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex) = a.log(Q) + b.log(Q)/Q^{0.2}+ c with prospective point and 2050
a = -0.069436354904   
b = -0.31259701454 
c = 7.8457420461

In [50]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap.head(1))
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a*np.log(1000*x_reg)+b*np.log(1000*x_reg)/((1000*x_reg)**0.2)+c)

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='SOEC Electrolysis - Prospective',
                         marker = dict(color=colour_soec, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='SOEC Electrolysis - Historical',
                         marker = dict(color=colour_soec, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='SOEC Electrolysis - Learning Curve',
                         line=dict(color=colour_soec, width=line_width)))

years = hist_cost.year.unique()
for i in range(len(years)):
    x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].installed_cap.head(1))
    y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].c_inv.head(1))
    year = years[i]
    fig.add_annotation(x=np.log10(x), y=np.log10(y),
            text=str(year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))


## PEM Electrolysis

In [51]:
# Historical data
hist_cost = hist_cost_electrolysis[hist_cost_electrolysis.tech == 'PEM_ELECTROLYSIS'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'PEM_ELECTROLYSIS'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex) = a*log(x) + b*(log(x))**c + d with prospective point and 2050
a = -0.031032067186 
b = 2.5252099604 
c = -0.73468149012
d = 6.351207017

In [52]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap)
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a*np.log(1000*x_reg)+b*(np.log(1000*x_reg))**c+d)

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='PEM Electrolysis - Prospective',
                         marker = dict(color=colour_pem, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='PEM Electrolysis - Historical',
                         marker = dict(color=colour_pem, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='PEM Electrolysis - Learning Curve',
                         line=dict(color=colour_pem, width=line_width)))

years = hist_cost.year.unique()
for i in range(len(years)):
    x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].installed_cap.sample(1))
    y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].c_inv.sample(1))
    year = years[i]
    fig.add_annotation(x=np.log10(x), y=np.log10(y),
            text=str(year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))



## ALK Electrolysis

In [53]:
# Historical data
hist_cost = hist_cost_electrolysis[hist_cost_electrolysis.tech == 'ALKALINE_ELECTROLYSIS'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'ALKALINE_ELECTROLYSIS'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex) = a + b.log(Q) with prospective points in 2030 and 2050
a = 6.8261573831
b = -0.10563795844

In [54]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap.head(1))
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))


# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         mode='markers',
                         marker_symbol='circle-open',
                         name='ALK Electrolysis - Prospective',
                         marker = dict(color=colour_alk, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         textfont_size=years_font_size,
                         name='ALK Electrolysis - Historical',
                         marker = dict(color=colour_alk, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         mode='lines',
                         name='ALK Electrolysis - Learning Curve',
                         line=dict(color=colour_alk, width=line_width)))

years = hist_cost.year.unique()
for i in range(len(years)):
    x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].installed_cap.sample(1))
    y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].c_inv.sample(1))
    year = years[i]
    fig.add_annotation(x=np.log10(x), y=np.log10(y),
            text=str(year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))

fig.show()

# Carbon Capture

In [55]:
# Historical data
hist_cost = hist_cost_carbon_capture[hist_cost_carbon_capture.tech == 'CARBON_CAPTURE'][['year','c_inv','installed_cap']]

# Prospective data 
future_cost = future_cost_data[future_cost_data.tech == 'CARBON_CAPTURE'][['year', 'cost_ratio', 'c_inv', 'installed_cap']]

# Learning curve : log(capex) = a + b.log(Q) with prospective points in 2030 and 2050
a = 4.7242033781 
b = -0.13316823733

In [56]:
hist_installed_cap = hist_cost['installed_cap']
hist_c_inv = hist_cost['c_inv']
hist_year = hist_cost['year']

future_installed_cap = future_cost['installed_cap']/1000
future_c_inv = future_cost['c_inv']

first_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.min()
last_hist_point_year = hist_cost[(hist_cost.c_inv >= 0)].year.max()
log_first_point = np.log10(hist_cost[(hist_cost.year == first_point_year)].installed_cap.head(1))
log_last_point = np.log10(future_cost[(future_cost.year == 2050)].installed_cap/1000)

x_reg = np.logspace(float(log_first_point), float(log_last_point), 100)
y_reg = np.exp(a+b*np.log(x_reg))

first_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].installed_cap)
first_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].c_inv)
first_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == first_point_year)].year)

last_x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].installed_cap)
last_y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].c_inv)
last_year = int(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == last_hist_point_year)].year)

prosp_x = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].installed_cap)/1000
prosp_y = float(future_cost[(future_cost.c_inv >= 0) & (future_cost.year == 2050)].c_inv)
prosp_year = 2050

# Add traces

fig.add_trace(go.Scatter(x=future_installed_cap, y=future_c_inv,
                         xaxis="x2",yaxis="y2",
                         mode='markers',
                         marker_symbol='circle-open',
                         legendgroup="group2",
                         legendgrouptitle_text="CO<sub>2</sub> capacity",
                         legendgrouptitle_font_color=colour_second_axis,
                         name='Carbon Capture - Prospective',
                         marker = dict(color=colour_carbon_capture, size=marker_size)))

fig.add_trace(go.Scatter(x=hist_installed_cap, y=hist_c_inv,
                         xaxis="x2",yaxis="y2",
                         #text = hist_year,
                         #textposition='top right',
                         mode='markers+text',
                         marker_symbol='circle',
                         #textfont_color = colour_carbon_capture,
                         textfont_size=years_font_size,
                         legendgroup="group2",
                         name='Carbon Capture - Historical',
                         marker = dict(color=colour_carbon_capture, size=marker_size)))

fig.add_trace(go.Scatter(x=x_reg, y=y_reg,
                         xaxis="x2",yaxis="y2",
                         mode='lines',
                         legendgroup="group2",
                         name='Carbon Capture - Learning Curve',
                         line=dict(color=colour_carbon_capture, width=line_width)))

fig.add_annotation(x=np.log10(prosp_x), y=np.log10(prosp_y),
                   xref="x2",yref="y2",
                   text=str(prosp_year),
                   showarrow=True,
                   font=dict(
                       family="Times New Roman",
                       size=years_font_size,color=colour_main_axis))


years = hist_cost.year.unique()
for i in range(len(years)):
    x = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].installed_cap.sample(1))
    y = float(hist_cost[(hist_cost.c_inv >= 0) & (hist_cost.year == years[i])].c_inv.sample(1))
    year = years[i]
    fig.add_annotation(x=np.log10(x), y=np.log10(y),
            xref="x2",yref="y2",
            text=str(year),
            showarrow=True,
            font=dict(
                family="Times New Roman",
                size=years_font_size,
                color=colour_main_axis))
    
fig['layout']['yaxis1']['showgrid'] = False
fig['layout']['xaxis1']['showgrid'] = False

# Final Graph

In [57]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 1200,
    'width': 1800,
    'scale': 3 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

fig.show(config=config)

In [58]:
# Download image
fig.write_image("pics/LR_graph_V2.pdf",width=1800, height=1200, scale=3)